In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from IPython.display import HTML
from matplotlib.dates import DateFormatter
from datetime import datetime
from datetime import timedelta

import plotly.offline as pyo
import plotly.graph_objs as go 
from plotly.subplots import make_subplots

In [2]:
# setting up disply options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Our world in data - import data
url = 'https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv?raw=true'

dfVaccination = pd.read_csv(url, 
                            encoding = 'utf-8', 
                            parse_dates=['date'],
                            sep = ',')
dfVaccination.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [4]:
dfVaccination.dtypes

iso_code                                         object
continent                                        object
location                                         object
date                                     datetime64[ns]
total_cases                                     float64
new_cases                                       float64
new_cases_smoothed                              float64
total_deaths                                    float64
new_deaths                                      float64
new_deaths_smoothed                             float64
total_cases_per_million                         float64
new_cases_per_million                           float64
new_cases_smoothed_per_million                  float64
total_deaths_per_million                        float64
new_deaths_per_million                          float64
new_deaths_smoothed_per_million                 float64
reproduction_rate                               float64
icu_patients                                    

In [5]:
dfVaccination.location.unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Asia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Europe', 'European Union', 'Faeroe Islands', 'Falkland Islands',
       'Fiji', 'Finland', 'Fran

In [6]:
# drop continents from data frame
countries = ['South America', 'North America', 'World', 'Oceania','Europe', 'International', 'Africa', 'Asia', 'European Union']
i = dfVaccination[dfVaccination.location.isin(countries)].index

In [7]:
dfVaccination.drop(i, inplace=True)

In [8]:
# Checking the max avaiable date
dfVaccination.date.max()

Timestamp('2021-04-09 00:00:00')

In [9]:
# Creating support columns for the dataframe
dfVaccination['new_cases_per_million_rollmean7'] = round(dfVaccination['new_cases_per_million'].rolling(7, center=True).mean(),2) # Rolling mean 7 days
dfVaccination['new_deaths_rollmean7'] = round(dfVaccination['new_deaths'].rolling(7, center=True).mean(),2) # Rolling mean 7 days
dfVaccination['total_tests_per_million'] = round((dfVaccination['total_tests']/dfVaccination['population'])*1000000,2)
dfVaccination['people_fully_vaccinated_per_million'] = round((dfVaccination['people_fully_vaccinated']/dfVaccination['population'])*1000000,2)

dfVaccination['people_fully_vaccinated_perc'] = round((dfVaccination['people_fully_vaccinated']/ dfVaccination['population'])*100,2)

dfVaccination['new_vaccinations_per_million'] =  round((dfVaccination['new_vaccinations']/dfVaccination['population'])*1000000,2)


In [10]:
dfVaccination.dtypes

iso_code                                         object
continent                                        object
location                                         object
date                                     datetime64[ns]
total_cases                                     float64
new_cases                                       float64
new_cases_smoothed                              float64
total_deaths                                    float64
new_deaths                                      float64
new_deaths_smoothed                             float64
total_cases_per_million                         float64
new_cases_per_million                           float64
new_cases_smoothed_per_million                  float64
total_deaths_per_million                        float64
new_deaths_per_million                          float64
new_deaths_smoothed_per_million                 float64
reproduction_rate                               float64
icu_patients                                    

In [11]:
# Plotagem de evolução de casos e média móvel centrada dos últimos
# 7 dias
def plotagem_media_movel(dfPlot:            pd.DataFrame,
                         datecolumn:        str,
                         qtdcolumn:         str,
                         smoothmeancolumn:  str,
                         title:             str):
  #-----------------------------------------------------------#
  # dfPlot: DataFrame com casos para o plot                   #
  # datecolumn: Nome da Coluna de Data do Daframe             #
  # qtdcolumn: Nome da Coluna as quantidades de casos         #
  # smoothmeancolumn: Coluna com a média móvel  de 7 dias     #
  # titulo: Título do Gráfico                                 #
  # Retorna a figura já plotada                               #
  #-----------------------------------------------------------#
  layout = go.Layout(
    xaxis_title_text = 'The side handlebars act as filters',
    template = 'plotly_white',
    title = { 'text': f'{title}<br>**Interactive Graph**', 
            'xanchor': 'center',
            'yanchor': 'top', 'y': 0.95, 'x': 0.45, 'font': dict(size=20)
          },
    xaxis=dict(
        title='Date',
        type='date',
        showgrid=False
    ),
    yaxis=dict(
        title='Cases - Daily '
    ),
    hovermode='closest',
  )

  trace1 = go.Bar(
      x=dfPlot[f'{datecolumn}'],
      y=dfPlot[f'{qtdcolumn}'],
      name='Daily'
  )

  trace2 = go.Scatter(
      x=dfPlot[f'{datecolumn}'],
      y=dfPlot[f'{smoothmeancolumn}'],
      mode='lines',
          marker=dict(
              color='Red',
              size=120,
              line=dict(
                  color='Red',
                  width=20
              )),
      name='Rolling Mean - 7 days'
  )

  data=[trace1, trace2]

  figure = go.Figure(data=data, layout=layout)

  figure['layout'].update(
      legend=dict(
          orientation="h",
          x=0,
          y=1.0
      ),
      xaxis=dict(
          rangeslider=dict(
              visible = True
          ),
          type='date',
      ),
  )

  figure.show()

In [12]:
to_plot = dfVaccination.loc[dfVaccination.location=='Brazil']

In [35]:
country1 = dfVaccination.loc[dfVaccination.location=='Afghanistan']
country2 = dfVaccination.loc[dfVaccination.location=='Afghanistan']


In [36]:
if country1.people_fully_vaccinated_per_million.isnull().all():
  country1.people_fully_vaccinated_per_million = round(((country1['new_vaccinations'].cumsum()/2)/country1['population'])*1000000,2)
  country1['people_fully_vaccinated_perc']     = round(((country1['new_vaccinations'].cumsum()/2)/country1['population'])*100,2)

if country2.people_fully_vaccinated_per_million.isnull().all():
  country2.people_fully_vaccinated_per_million =  round(((country2['new_vaccinations'].cumsum()/2)/country2['population'])*1000000,2)
  country2['people_fully_vaccinated_perc']     = round(((country2['new_vaccinations'].cumsum()/2)/country2['population'])*100,2)  

C:\ProgramData\Anaconda3\envs\streamlit_COVID_VACCINATION\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-929dc0d67eac>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-929dc0d67eac>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [16]:
country1.tail(2)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,new_cases_per_million_rollmean7,new_deaths_rollmean7,total_tests_per_million,people_fully_vaccinated_per_million,people_fully_vaccinated_perc,new_vaccinations_per_million
10717,BRA,South America,Brazil,2021-04-08,13279857.0,86652.0,62859.0,345025.0,4249.0,2820.143,62475.978,407.660,295.724,1623.193,19.990,13.268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24981199.0,19386057.0,5595142.0,783203.0,726681.0,11.75,9.12,2.63,3419.0,NaN,212559409.0,25.04,33.5,8.552,5.06,14103.452,3.4,177.961,8.11,10.1,17.9,NaN,2.2,75.88,0.765,261.24,NaN,NaN,26322.72,2.63,3684.63
10718,BRA,South America,Brazil,2021-04-09,13373174.0,93317.0,66156.0,348718.0,3693.0,2930.286,62914.994,439.016,311.235,1640.567,17.374,13.786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25735314.0,19886870.0,5848444.0,754115.0,728920.0,12.11,9.36,2.75,3429.0,NaN,212559409.0,25.04,33.5,8.552,5.06,14103.452,3.4,177.961,8.11,10.1,17.9,NaN,2.2,75.88,0.765,245.25,NaN,NaN,27514.40,2.75,3547.78


In [17]:
country1.people_fully_vaccinated_per_million.isna().unique()

array([ True, False])

In [18]:

country1.people_fully_vaccinated_perc.isna().unique()

array([ True, False])

In [19]:

country1['new_deaths'].sum()/country1['new_cases'].sum()*100





today = pd.to_datetime('today')

last_month = today - pd.DateOffset(months=1)








last_month_cases = country1.loc[country1['date'] < last_month, 'new_cases'].sum()

this_month_cases = country1.loc[country1['date'] >= last_month, 'new_cases'].sum()

growth = -100 - round(((this_month_cases-last_month_cases)/last_month_cases)*100,2)
Mortality_Rate1         = country1['new_deaths'].sum()/country1['new_cases'].sum()*100

growth =  round(abs(1+(this_month_cases-last_month_cases)/last_month_cases)*100,2)

print(growth)
print(Mortality_Rate1)
#print(last_month_deaths)
#print(this_month_deaths)



19.38
2.6075933805991007


In [20]:
plotagem_media_movel(to_plot, 
                     'date', 
                     'new_cases', 
                     'new_cases_per_million_rollmean7',	
                     'Cases Mean')

In [21]:
  Vaccination_Start_Date1 = country1.loc[~country1['new_vaccinations'].isnull()].iloc[0]['date']
  Vaccination_Start_Date1 = Vaccination_Start_Date1.strftime("%m/%d/%y")
  
  Vaccination_Start_Date1

'01/17/21'

In [22]:
Vaccination_Start_Date1

'01/17/21'

In [54]:
# Função para plotar painel de Casos de COVID-19, óbitos por COVID-19
# e Percentual de Isolamento.

def COVID_Cases_per_million(dfCountry1: pd.DataFrame, 
                            dfCountry2: pd.DataFrame,
                            cTitleCountry1: str,
                            cTitleCountry2: str,
                            titulo: str):
  #-----------------------------------------------------------#
  # dfCountry1: Country 1 DataFrame                           #
  # dfCountry2: Country 2 DataFrame                           #
  # cTitleCountry1: selected country1                         #
  # cTitleCountry1: selected country2                         #
  # titulo: Graph Title                                       #
  # Returns: Ploted Graph                                     #
  #-----------------------------------------------------------#

  fig = make_subplots(rows=3, 
                      cols=2, 
                      vertical_spacing=0.15,
                      horizontal_spacing=0.08,
                      specs=[[{"type": "scatter"}, {"type": "scatter"}],
                             [{"type": "scatter"}, {"type": "scatter"}],                          
                             [{"type": "table"}, {"type": "table"}]],
                      subplot_titles=(f"<b>Daily cases per million in {cTitleCountry1}</b>",
                                      f"<b>Daily cases per million in {cTitleCountry2}</b>",
                                      f"<b>Testing and Vaccination Evolution per million in {cTitleCountry1}</b>",     
                                      f"<b>Testing and Vaccination Evolution per million in {cTitleCountry2}</b>",
                                      f"<b>Demographics of {cTitleCountry1}</b>",
                                      f"<b>Demographics of {cTitleCountry2}</b>"))
                                              
  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['new_cases_per_million'],
                marker=dict(
                            color='deepskyblue',
                            size=120
          ),
      name=f"Daily cases per million in {cTitleCountry1}",
      hovertemplate=
        f"<b>{cTitleCountry1}'s new cases per million</b><br><br>" +
        "Date: %{x}<br>" +
        "New cases per million: %{y:,.}<br>" +
        "<extra></extra>"),
      row=1, col=1
  )

  fig.add_trace(
    go.Scatter(
    x=dfCountry1['date'],
    y=dfCountry1['new_cases_per_million_rollmean7'],
    mode='lines',
        marker=dict(
            color='Red',
            size=120,
            line=dict(
                color='Red',
                width=20
            )),
    name=f"{cTitleCountry1}'s 7 day rolling mean",
    hovertemplate=
        f"<b>{cTitleCountry1}'s 7 day rolling mean</b><br><br>" +
        "Date: %{x}<br>" +
        "New cases: %{y:,.}<br>" +
        "<extra></extra>"),
    row=1, col=1)

  fig.add_trace(
      go.Scatter(x=dfCountry2['date'], 
                y=dfCountry2['new_cases_per_million'],
                marker=dict(
                            color='gold',
                            size=120,
          ), 
      name=f"Daily Cases in {cTitleCountry2}",
      hovertemplate=
        f"<b>{cTitleCountry2}'s new cases per million </b><br><br>" +
        "Date: %{x}<br>" +
        "New cases per million: %{y:,.}<br>" +
        "<extra></extra>"),      
      row=1, col=2
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['new_cases_per_million_rollmean7'],
      mode='lines',
          marker=dict(
              color='Red',
              size=120,
              line=dict(
                  color='Red',
                  width=20
              )),
      name=f"{cTitleCountry2}'s 7 day rolling mean",
      hovertemplate=
        f"<b>{cTitleCountry2}'s 7 day rolling mean</b><br><br>" +
        "Date: %{x}<br>" +
        "New cases per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=1, col=2)
    
  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['people_fully_vaccinated_per_million'],
                marker=dict(color='Purple',
                            size=160,
                            line=dict(
                            color='Purple',
                            width=40)), 
      name="People Fully Vaccinated per Million",
      hovertemplate=
        f"<b>{cTitleCountry1}'s fully vaccinated per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>"),   
      row=2, col=1
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry1['date'],
      y=dfCountry1['new_vaccinations'],
      mode='lines',
          marker=dict(
              color='darkorchid',
              size=160,
              line=dict(
                  color='darkorchid',
                  width=40
              )),
      name='New Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry1}'s vaccinated per million<br>(at least one dose) </b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=1)

  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['total_tests_per_million'],
                marker=dict(
                            color='Green',
                            size=180,
                            line=dict(
                            color='Green',
                            width=40)),
      name="People Tested per Million", 
      hovertemplate=
        f"<b>{cTitleCountry1}'s tested per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Tested per million: %{y:,.}<br>" +
        "<extra></extra>"),
      row=2, col=1
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['new_vaccinations'],
      mode='lines',
          marker=dict(
              color='darkorchid',
              size=160,
              line=dict(
                  color='darkorchid',
                  width=40
              )),
      name='People Fully Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry2}'s vaccinated per million<br>(at least one dose) </b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['people_fully_vaccinated_per_million'],
      mode='lines',
          marker=dict(
              color='Purple',
              size=160,
              line=dict(
                  color='Purple',
                  width=40
              )),
      name='People Fully Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry2}'s fully vaccinated per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['total_tests_per_million'],
      mode='lines',
          marker=dict(
              color='Green',
              size=160,
              line=dict(
                  color='Green',
                  width=40
              )),
      name='Tested per Million',
      hovertemplate=
        f"<b>{cTitleCountry2}'s tested per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Tested per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)  

  # Table Calculations for the first country
  today = pd.to_datetime('today')
  last_month = today - pd.DateOffset(months=1)
  last_month_cases1 = dfCountry1.loc[dfCountry1['date'] < last_month, 'new_cases'].sum()
  this_month_cases1 = dfCountry1.loc[dfCountry1['date'] >= last_month, 'new_cases'].sum()
  Population1             = dfCountry1['population'].max()
  Perc_Fully_Vaccinated1  = dfCountry1['people_fully_vaccinated_perc'].max()#round((dfCountry1['people_fully_vaccinated'].max()/ 
                           # dfCountry1['population'].max())*100,2)
  try:                            
    Vaccination_Start_Date1 = dfCountry1.loc[~dfCountry1['new_vaccinations'].isnull()].iloc[0]         ['date']
  except:
    Vaccination_Start_Date1 = '-'

  Cases_Total1            = int(dfCountry1['total_cases'].max())
  Deaths_Total1           = int(dfCountry1['total_deaths'].max())
  Mortality_Rate1         = round(dfCountry1['new_deaths'].sum()/dfCountry1['new_cases'].sum()*100,2)
  Growth_Rate1            = round(abs(1+(this_month_cases1-last_month_cases1)/last_month_cases1)*100,2)
  cells1 = []
  cells1.append(round((Population1/1000000),1))
  cells1.append(f'{Perc_Fully_Vaccinated1}%')

  if Vaccination_Start_Date1 == '-':
    cells1.append(Vaccination_Start_Date1)
  else:
    cells1.append(Vaccination_Start_Date1.strftime("%m/%d/%y"))

  cells1.append("{:,}".format(Cases_Total1))
  cells1.append("{:,}".format(Deaths_Total1))
  cells1.append(f'{Mortality_Rate1}%')
  cells1.append(f'{Growth_Rate1}%')      

  fig.add_trace(
      go.Table(
          header=dict(
              values=["<b>Population (Millions)</b>", "<b>Perc. of Population fully vaccineted</b>","<b>Vaccination started on (MM/DD/YY)</b>", "<b>Cases Total</b>", "<b>Deaths Total</b>", "<b>Mortality Rate</b>", "<b>Last Month Growth Rate</b>"],
              font=dict(size=10),
              align="center"
          ),
          cells=dict(
              values=cells1,
              align = "center")
      ),
      row=3, col=1
  )          

  # Table Calculations for the second country
  today = pd.to_datetime('today')
  last_month = today - pd.DateOffset(months=1)
  last_month_cases2 = dfCountry2.loc[dfCountry2['date'] < last_month, 'new_cases'].sum()
  this_month_cases2 = dfCountry2.loc[dfCountry2['date'] >= last_month, 'new_cases'].sum()
  Population2             = dfCountry2['population'].max()
  Perc_Fully_Vaccinated2  = round((dfCountry2['people_fully_vaccinated'].max()/ 
                            dfCountry2['population'].max())*100,2)
  try:                            
    Vaccination_Start_Date2 = dfCountry2.loc[~dfCountry2['new_vaccinations'].isnull()].iloc[0]
    ['date']
  except:
    Vaccination_Start_Date2 = '-'

  Cases_Total2            = int(dfCountry2['total_cases'].max())
  Deaths_Total2           = int(dfCountry2['total_deaths'].max())
  Mortality_Rate2         = round(dfCountry2['new_deaths'].sum()/dfCountry2['new_cases'].sum()*100,2)
  Growth_Rate2            = round(abs(1+(this_month_cases2-last_month_cases2)/last_month_cases2)*100,2)
  cells2 = []
  cells2.append(round((Population2/1000000),1))
  cells2.append(f'{Perc_Fully_Vaccinated2}%')
  
  if Vaccination_Start_Date2 == '-':
    cells2.append(Vaccination_Start_Date2)
  else:
    cells2.append(Vaccination_Start_Date2.strftime("%m/%d/%y"))

  cells2.append("{:,}".format(Cases_Total2))
  cells2.append("{:,}".format(Deaths_Total2))
  cells2.append(f'{Mortality_Rate2}%')
  cells2.append(f'{Growth_Rate2}%')

  fig.add_trace(
      go.Table(
          header=dict(
              values=["<b>Population (Millions)</b>", "<b>Perc of Population fully vaccineted</b>","<b>Vaccination started on (MM/DD/YY)</b>", "<b>Cases Total</b>", "<b>Deaths Total</b>", "<b>Mortality Rate</b>", "<b>Last Month Growth Rate</b>"],
              font=dict(size=10),
              align="center"
          ),

          cells=dict(
              values=cells2,
              align = "center")
      ),
      row=3, col=2
  ) 

  data_date = dfCountry1.date.max().strftime("%m/%d/%Y")
  plot1 = ''
  plot2 = ''

  if dfCountry1['total_tests_per_million'].isnull().values.sum():
     plot1 = f'** No all test data avaiable for {cTitleCountry1}'
  elif dfCountry2['total_tests_per_million'].isnull().values.sum():
     plot2 = f'** No all test data avaiable for {cTitleCountry2}'

  fig.update_layout(
      template = 'plotly_white',
      title = { 'text': f'{titulo}<br>**Interative Graph**',
                'font': dict(size=20),
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
      showlegend=False,        
      yaxis = {  'categoryorder': 'total descending'},
      yaxis3 = { 'categoryorder': 'total descending'},              
      autosize=False,
      width=1280,
      height=900,     
      margin=dict(
          l=50,
          r=50,
          b=10,
          t=110,
          pad=2
      )
  )

  fig.add_annotation(text=f'Source: https://ourworldindata.org/ <br> * Data available until: {data_date} <br> {plot1} <br> {plot2}',
                  xref="paper", yref="paper",
                  x=0.9999, y=0.001, showarrow=False)

  if not plot1 is None: 
    fig.add_annotation(text=f'<b>{plot1}</b>',
                    xref="paper", yref="paper",
                    x=0.03, y=0.33, showarrow=False)                        

  if not plot2 is None: 
    fig.add_annotation(text=f'<b>{plot2}</b>',
                    xref="paper", yref="paper",
                    x=0.77, y=0.33, showarrow=False)               

  fig.show()
  return 

In [49]:
try:
  country1.loc[~country1['new_vaccinations'].isnull()].iloc[0]['date']
except:
  print("An exception occurred")


An exception occurred


In [25]:
country2['total_tests_per_million'].isnull().values.any()

True

In [26]:
country1.people_fully_vaccinated.isnull().unique()

array([ True, False])

In [27]:
country1.tail()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,new_cases_per_million_rollmean7,new_deaths_rollmean7,total_tests_per_million,people_fully_vaccinated_per_million,people_fully_vaccinated_perc,new_vaccinations_per_million
10714,BRA,South America,Brazil,2021-04-05,13013601.0,28645.0,62855.143,332752.0,1319.0,2698.000,61223.359,134.762,295.706,1565.454,6.205,12.693,0.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21960953.0,17128933.0,4832020.0,890825.0,554114.0,10.33,8.06,2.27,2607.0,NaN,212559409.0,25.04,33.5,8.552,5.06,14103.452,3.4,177.961,8.11,10.1,17.9,NaN,2.2,75.88,0.765,295.72,2820.14,NaN,22732.56,2.27,4190.95
10715,BRA,South America,Brazil,2021-04-06,13100580.0,86979.0,63210.143,336947.0,4195.0,2757.286,61632.558,409.199,297.376,1585.190,19.736,12.972,0.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22850791.0,17791338.0,5059453.0,889838.0,661176.0,10.75,8.37,2.38,3111.0,NaN,212559409.0,25.04,33.5,8.552,5.06,14103.452,3.4,177.961,8.11,10.1,17.9,NaN,2.2,75.88,0.765,311.24,2930.29,NaN,23802.54,2.38,4186.30
10716,BRA,South America,Brazil,2021-04-07,13193205.0,92625.0,63494.000,340776.0,3829.0,2751.571,62068.318,435.761,298.712,1603.204,18.014,12.945,0.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24197996.0,18859274.0,5338722.0,1347205.0,720288.0,11.38,8.87,2.51,3389.0,NaN,212559409.0,25.04,33.5,8.552,5.06,14103.452,3.4,177.961,8.11,10.1,17.9,NaN,2.2,75.88,0.765,282.32,NaN,NaN,25116.38,2.51,6338.02
10717,BRA,South America,Brazil,2021-04-08,13279857.0,86652.0,62859.000,345025.0,4249.0,2820.143,62475.978,407.660,295.724,1623.193,19.990,13.268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24981199.0,19386057.0,5595142.0,783203.0,726681.0,11.75,9.12,2.63,3419.0,NaN,212559409.0,25.04,33.5,8.552,5.06,14103.452,3.4,177.961,8.11,10.1,17.9,NaN,2.2,75.88,0.765,261.24,NaN,NaN,26322.72,2.63,3684.63
10718,BRA,South America,Brazil,2021-04-09,13373174.0,93317.0,66156.000,348718.0,3693.0,2930.286,62914.994,439.016,311.235,1640.567,17.374,13.786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25735314.0,19886870.0,5848444.0,754115.0,728920.0,12.11,9.36,2.75,3429.0,NaN,212559409.0,25.04,33.5,8.552,5.06,14103.452,3.4,177.961,8.11,10.1,17.9,NaN,2.2,75.88,0.765,245.25,NaN,NaN,27514.40,2.75,3547.78


In [55]:
COVID_Cases_per_million(country1, 
                        country2,
                        'Brazil',
                        'United States of America',
                        'COVID-19 Country Comparison')

In [34]:
# Função para plotar painel de Casos de COVID-19, óbitos por COVID-19
# e Percentual de Isolamento.

def COVID_deaths (dfCountry1: pd.DataFrame, 
                  dfCountry2: pd.DataFrame,
                  cTitleCountry1: str,
                  cTitleCountry2: str,
                  titulo: str):
  #-----------------------------------------------------------#
  # dfCountry1: Country 1 DataFrame                           #
  # dfCountry2: Country 2 DataFrame                           #
  # cTitleCountry1: selected country1                         #
  # cTitleCountry1: selected country2                         #
  # titulo: Graph Title                                       #
  # Returns: Ploted Graph                                     #
  #-----------------------------------------------------------#

  fig = make_subplots(rows=3, 
                      cols=2, 
                      vertical_spacing=0.15,
                      horizontal_spacing=0.08,
                      specs=[[{"type": "scatter"}, {"type": "scatter"}],
                             [{"type": "scatter"}, {"type": "scatter"}],                          
                             [{"type": "table"}, {"type": "table"}]],
                      subplot_titles=(f"<b>Daily death count in {cTitleCountry1}</b>",
                                      f"<b>Daily death count in {cTitleCountry2}</b>",
                                      f"<b>Testing and Vaccination Evolution per million in {cTitleCountry1}</b>",     
                                      f"<b>Testing and Vaccination Evolution per million in {cTitleCountry2}</b>",
                                      f"<b>Demographics of {cTitleCountry1}</b>",
                                      f"<b>Demographics of {cTitleCountry2}</b>"))
                                              
  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['new_deaths'],
                marker=dict(
                            color='deepskyblue',
                            size=120
          ),
      name=f"Daily death count in {cTitleCountry1}",
      hovertemplate=
        f"<b>{cTitleCountry1}'s daily death count </b><br><br>" +
        "Date: %{x}<br>" +
        "Daily death count: %{y:,.}<br>" +
        "<extra></extra>"),
      row=1, col=1
  )

  fig.add_trace(
    go.Scatter(
    x=dfCountry1['date'],
    y=dfCountry1['new_deaths_rollmean7'],
    mode='lines',
        marker=dict(
            color='Red',
            size=120,
            line=dict(
                color='Red',
                width=20
            )),
    name=f"{cTitleCountry1}'s 7 day rolling mean",
    hovertemplate=
        f"<b>{cTitleCountry1}'s 7 day rolling mean</b><br><br>" +
        "Date: %{x}<br>" +
        "Daily death count: %{y:,.}<br>" +
        "<extra></extra>"),
    row=1, col=1)

  fig.add_trace(
      go.Scatter(x=dfCountry2['date'], 
                y=dfCountry2['new_deaths'],
                marker=dict(
                            color='gold',
                            size=120,
          ), 
      name=f"Daily death count in {cTitleCountry2}",
      hovertemplate=
        f"<b>{cTitleCountry2}'s daily death count</b><br><br>" +
        "Date: %{x}<br>" +
        "Daily death count: %{y:,.}<br>" +
        "<extra></extra>"),      
      row=1, col=2
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['new_deaths_rollmean7'],
      mode='lines',
          marker=dict(
              color='Red',
              size=120,
              line=dict(
                  color='Red',
                  width=20
              )),
      name=f"{cTitleCountry2}'s 7 day rolling mean",
      hovertemplate=
        f"<b>{cTitleCountry2}'s 7 day rolling mean</b><br><br>" +
        "Date: %{x}<br>" +
        "Daily death count: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=1, col=2)
    
  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['people_fully_vaccinated_per_million'],
                marker=dict(color='Purple',
                            size=160,
                            line=dict(
                            color='Purple',
                            width=40)), 
      name="People Fully Vaccinated per Million",
      hovertemplate=
        f"<b>{cTitleCountry1}'s fully vaccinated per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>"),   
      row=2, col=1
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry1['date'],
      y=dfCountry1['new_vaccinations_per_million'],
      mode='lines',
          marker=dict(
              color='darkorchid',
              size=160,
              line=dict(
                  color='darkorchid',
                  width=40
              )),
      name='New Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry1}'s vaccinated per million<br>(at least one dose) </b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=1)

  fig.add_trace(
      go.Scatter(x=dfCountry1['date'], 
                y=dfCountry1['total_tests_per_million'],
                marker=dict(
                            color='Green',
                            size=180,
                            line=dict(
                            color='Green',
                            width=40)),
      name="People Tested per Million", 
      hovertemplate=
        f"<b>{cTitleCountry1}'s tested per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Tested per million: %{y:,.}<br>" +
        "<extra></extra>"),
      row=2, col=1
  )

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['new_vaccinations_per_million'],
      mode='lines',
          marker=dict(
              color='darkorchid',
              size=160,
              line=dict(
                  color='darkorchid',
                  width=40
              )),
      name='People Fully Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry2}'s vaccinated per million<br>(at least one dose) </b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['people_fully_vaccinated_per_million'],
      mode='lines',
          marker=dict(
              color='Purple',
              size=160,
              line=dict(
                  color='Purple',
                  width=40
              )),
      name='People Fully Vaccinated per Million"',
      hovertemplate=
        f"<b>{cTitleCountry2}'s fully vaccinated per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Vaccinated per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)

  fig.add_trace(
      go.Scatter(
      x=dfCountry2['date'],
      y=dfCountry2['total_tests_per_million'],
      mode='lines',
          marker=dict(
              color='Green',
              size=160,
              line=dict(
                  color='Green',
                  width=40
              )),
      name='Tested per Million',
      hovertemplate=
        f"<b>{cTitleCountry2}'s tested per million</b><br><br>" +
        "Date: %{x}<br>" +
        "Tested per million: %{y:,.}<br>" +
        "<extra></extra>",
      showlegend=False),
      row=2, col=2)  

  # Table Calculations for the first country
  today = pd.to_datetime('today')
  last_month = today - pd.DateOffset(months=1)

  last_month_death1 = dfCountry1.loc[dfCountry1['date'] < last_month, 'new_deaths'].sum()
  this_month_death1 = dfCountry1.loc[dfCountry1['date'] >= last_month, 'new_deaths'].sum()
  Population1             = dfCountry1['population'].max()
  Perc_Fully_Vaccinated1  = dfCountry1['people_fully_vaccinated_perc'].max()#round((dfCountry1['people_fully_vaccinated'].max()/ 
                           # dfCountry1['population'].max())*100,2)
  try:                          
    Vaccination_Start_Date1 = dfCountry1.loc[~dfCountry1['new_vaccinations'].isnull()].iloc[0]         ['date']
  except:
    Vaccination_Start_Date1 = '-'    

  casess_Total1           = int(dfCountry1['total_deaths'].max())
  deaths_Total1           = int(dfCountry1['total_deaths'].max())
  Mortality_Rate1         = round(dfCountry1['new_deaths'].sum()/dfCountry1['new_cases'].sum()*100,2)
  Growth_Rate1            = round(abs(1+(this_month_death1-last_month_death1)/last_month_death1)*100,2)
  cells1 = []
  cells1.append(round((Population1/1000000),1))
  cells1.append(f'{Perc_Fully_Vaccinated1}%')

  cells1.append(Vaccination_Start_Date1.strftime("%m/%d/%y"))
  cells1.append("{:,}".format(deaths_Total1))
  cells1.append("{:,}".format(deaths_Total1))
  cells1.append(f'{Mortality_Rate1}%')
  cells1.append(f'{Growth_Rate1}%')      

  fig.add_trace(
      go.Table(
          header=dict(
              values=["<b>Population (Millions)</b>", "<b>Perc. of Population fully vaccineted</b>","<b>Vaccination started on (MM/DD/YY)</b>", "<b>death count Total</b>", "<b>death count Total</b>", "<b>Mortality Rate</b>", "<b>Last Month Growth Rate</b>"],
              font=dict(size=10),
              align="center"
          ),
          cells=dict(
              values=cells1,
              align = "center")
      ),
      row=3, col=1
  )          

  # Table Calculations for the second country
  last_month_death2       = dfCountry2.loc[dfCountry2['date'] < last_month, 'new_deaths'].sum()
  this_month_death2       = dfCountry2.loc[dfCountry2['date'] >= last_month, 'new_deaths'].sum()
  Population2             = dfCountry2['population'].max()
  Perc_Fully_Vaccinated2  = round((dfCountry2['people_fully_vaccinated'].max()/ 
                            dfCountry2['population'].max())*100,2)
                            
  try:                          
    Vaccination_Start_Date2 = dfCountry2.loc[~dfCountry2['new_vaccinations'].isnull()].iloc[0]         ['date']
  except:
    Vaccination_Start_Date2 = '-'

  cases_Total2            = int(dfCountry2['total_cases'].max())
  deaths_Total2           = int(dfCountry2['total_deaths'].max())
  Mortality_Rate2         = round(dfCountry2['new_deaths'].sum()/dfCountry2['new_cases'].sum()*100,2)
  Growth_Rate2            = round(abs(1+(this_month_death2 - last_month_death2)/last_month_death2)*100,2)
  cells2 = []
  cells2.append(round((Population2/1000000),1))
  cells2.append(f'{Perc_Fully_Vaccinated2}%')

  if Vaccination_Start_Date2 == '-':
    cells2.append(Vaccination_Start_Date2
  else:
    cells2.append(Vaccination_Start_Date2.strftime("%m/%d/%y"))
    
  cells2.append("{:,}".format(cases_Total2))
  cells2.append("{:,}".format(deaths_Total2))
  cells2.append(f'{Mortality_Rate2}%')
  cells2.append(f'{Growth_Rate2}%')

  fig.add_trace(
      go.Table(
          header=dict(
              values=["<b>Population (Millions)</b>", "<b>Perc of Population fully vaccineted</b>","<b>Vaccination started on (MM/DD/YY)</b>", "<b>death count Total</b>", "<b>death count Total</b>", "<b>Mortality Rate</b>", "<b>Last Month Growth Rate</b>"],
              font=dict(size=10),
              align="center"
          ),

          cells=dict(
              values=cells2,
              align = "center")
      ),
      row=3, col=2
  ) 

  data_date = dfCountry1.date.max().strftime("%m/%d/%Y")
  plot1 = ''
  plot2 = ''

  if dfCountry1['total_tests_per_million'].isnull().values.sum():
     plot1 = f'** No all test data avaiable for {cTitleCountry1}'
  elif dfCountry2['total_tests_per_million'].isnull().values.sum():
     plot2 = f'** No all test data avaiable for {cTitleCountry2}'

  fig.update_layout(
      template = 'plotly_white',
      title = { 'text': f'{titulo}<br>**Interative Graph**',
                'font': dict(size=20),
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
      showlegend=False,        
      yaxis = {  'categoryorder': 'total descending'},
      yaxis3 = { 'categoryorder': 'total descending'},              
      autosize=False,
      width=1280,
      height=900,     
      margin=dict(
          l=50,
          r=50,
          b=10,
          t=110,
          pad=2
      )
  )

  fig.add_annotation(text=f'Source: https://ourworldindata.org/ <br> * Data available until: {data_date} <br> {plot1} <br> {plot2}',
                  xref="paper", yref="paper",
                  x=0.9999, y=0.001, showarrow=False)

  if not plot1 is None: 
    fig.add_annotation(text=f'<b>{plot1}</b>',
                    xref="paper", yref="paper",
                    x=0.03, y=0.33, showarrow=False)                        

  if not plot2 is None: 
    fig.add_annotation(text=f'<b>{plot2}</b>',
                    xref="paper", yref="paper",
                    x=0.77, y=0.33, showarrow=False)               

  fig.show()
  return 

In [30]:
COVID_deaths(country1, 
             country2,
            'Brazil',
            'United States of America',
            'COVID-19 Country Comparison')